## Active Learning

Download the titanic dataset here: https://drive.google.com/file/d/0Bz9_0VdXvv9bbVhpOEMwUDJ2elU/view?usp=sharing

In this exercise, we will simulate active learning. We will keep the small sample of observations for testing and we will test how quality of the model rises when we use active learning to choose labeled observations.

In [1]:
# Load the Data into variable df
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from operator import itemgetter 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import random
import copy

In [ ]:
filepath = 'C:/Users/Tim/Desktop/lighthouse/w8/we/'
file = 'titanic.csv'
df = pd.read_csv(filepath+file)

In [ ]:
# TEST SAMPLE
# USE THIS SAMPLE ONLY FOR TESTING
test_df = df.sample(n=100, random_state=42)
# KEEP ONLY THOSE WHO ARE NOT IN THE TEST SET
df = df[~df.PassengerId.isin(test_df.PassengerId.tolist())]

In [ ]:
# FIT THE FIRST MODEL ONLY ON THE DATAFRAME START_DF
start_df = df.sample(n=100, random_state=42)
# DROP OBS FROM START_DF FROM DF
df = df[~df.PassengerId.isin(start_df.PassengerId.tolist())]

In [ ]:
# X_start = start_df.drop(columns='survived')
# y_start = start_df['survived']
# X_test = test_df.drop(columns='survived')
# y_test = test_df['survived']

In [ ]:
clean_data = copy.deepcopy(start_df)

In [ ]:
def clean(data):
    new_data = copy.deepcopy(data)
    new_data.drop(['PassengerId','Name','Ticket','Cabin'], axis =1, inplace = True)
    new_data['Age'].fillna(np.mean(new_data['Age']), inplace = True)
    new_data['Age'].fillna(new_data['Embarked'].mode()[0], inplace = True)
    le = LabelEncoder()
    new_data['Sex'] = le.fit_transform(new_data['Sex'])
    new_data['Embarked'] = le.fit_transform(new_data['Embarked'].astype(str))
    num_feats = new_data[['Age','Fare']]
    scaled = StandardScaler().fit_transform(num_feats)
    new_data['Age'] = scaled[:,0]
    new_data['Fare'] = scaled[:,1]
    return(new_data)

In [ ]:
clean_data = clean(clean_data)

In [ ]:
X = clean_data.drop('Survived', axis = 1)
y = clean_data['Survived']

### Tasks

1. fit the first model only on the **start_df** using **SVM** and evaluate accuracy, precision and recall on test_df
2. in each iteration, add 10 observations from **df** to your trainset (choose the observation using active learning approach) 
    - score all observations in df and take 10 where the model isn't sure what class it is. The probability of surviving will be around 50% 
3. refit the model and evaluate on **test_df** again.    
3. the goal is to converge to the optimal solution as fast as possible by choosing **right** observations in each iteration
4. plot the graphs for each eval metric, where on the axis x is iteration number, on y is the metric value for that model

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
# x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state=123)

In [ ]:
# def model(model,X_train,y_train,X_test,y_test):
#     model = model
#     model.fit(X_train,y_train)
#     y_pred = model.predict(X_test)
#     print(metrics.accuracy(y_pred,y_test))
#     print(metrics.classification_report(y_pred,y_test))

In [ ]:
svc = SVC(probability = True)
svc.fit(X,y)

In [ ]:
y_pred = svc.predict((clean(test_df).drop(['Survived'],axis = 1)))
y_proba = svc.predict_proba((clean(test_df).drop(['Survived'],axis = 1)))
y_pred

In [ ]:
min(y_proba[0])

In [ ]:
print(accuracy_score(test_df['Survived'],y_pred))
print(recall_score(test_df['Survived'],y_pred))
print(precision_score(test_df['Survived'],y_pred))

# active

In [ ]:
data = pd.read_csv(filepath+file, header = 0)
df = copy.deepcopy(data)
test_df = df.sample(n=100, random_state=42)
df = df[~df.PassengerId.isin(test_df.PassengerId.tolist())]
start_df = df.sample(n=100, random_state=42)
df = df[~df.PassengerId.isin(start_df.PassengerId.tolist())]

In [ ]:
clean_data = copy.deepcopy(start_df)
clean_data = clean(clean_data)

In [ ]:
iterations = 10
batch = 10
metrics = []
svc = SVC(probability = True)

In [ ]:
for i in range(iterations):
    X = clean_data.drop('Survived', axis = 1)
    y = clean_data['Survived']
    svc.fit(X,y)
    y_pred = svc.predict((clean(test_df).drop(['Survived'],axis = 1)))
    acc = accuracy_score(test_df['Survived'],y_pred)
    recall = recall_score(test_df['Survived'],y_pred)
    precision = precision_score(test_df['Survived'],y_pred)
    met = [acc,recall,precision]
    metrics.append(met)
    
    temp_preds = svc.predict_proba((clean(df).drop(['Survived'],axis = 1)))
    margin_sample = abs(temp_preds[:,0] - temp_preds[:,1])
    to_get = list(itemgetter(*list(margin_sample.argsort()[:batch]))(df.index.values.tolist()))
    #to_get = list(np.argpartition(margin_sample, 5)[:5])
    #print(to_get)
    clean_data = pd.concat([clean_data,clean(df.loc[to_get,:])])
    
    df.drop(to_get, inplace = True)

In [ ]:
temp = pd.DataFrame(metrics)
temp = temp.rename(columns={0: 'accuracy',1:'recall',2:'precision'})

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
temp.accuracy.plot(legend=True)
temp.recall.plot(legend=True) 
temp.precision.plot(legend=True)
plt.xlabel('iteration')
plt.show()